# Description

This notebook is similar to `30` and `35`, but here I use the LVs that we found to be significantly enriched for the lipids CRISPR analysis, which might or might not coincide with the previously used LVs (those that discriminate clusters).
The traits here are from PhenomeXcan.

# Environment variables

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import conf

In [ ]:
N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

In [ ]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

# Modules

In [ ]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm

from gls import GLSPhenoplier

# Settings

In [ ]:
OUTPUT_DIR = conf.RESULTS["GLS"]
display(OUTPUT_DIR)

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
OUTPUT_FILENAME = OUTPUT_DIR / "gls_phenotypes-crispr_lvs-phenomexcan.pkl"
display(OUTPUT_FILENAME)

# Load data

## PhenomeXcan (S-MultiXcan)

In [ ]:
INPUT_SUBSET = "z_score_std"

In [ ]:
INPUT_STEM = "projection-smultixcan-efo_partial-mashr-zscores"

In [ ]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    f"{INPUT_SUBSET}-{INPUT_STEM}.pkl",
).resolve()

In [ ]:
data = pd.read_pickle(input_filepath)

In [ ]:
data.shape

In [ ]:
data.head()

## Clustering results

In [ ]:
CONSENSUS_CLUSTERING_DIR = Path(
    conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering"
).resolve()

display(CONSENSUS_CLUSTERING_DIR)

In [ ]:
input_file = Path(CONSENSUS_CLUSTERING_DIR, "best_partitions_by_k.pkl").resolve()
display(input_file)

In [ ]:
best_partitions = pd.read_pickle(input_file)

In [ ]:
best_partitions.shape

In [ ]:
best_partitions.head()

## MultiPLIER summary

In [ ]:
multiplier_model_summary = pd.read_pickle(conf.MULTIPLIER["MODEL_SUMMARY_FILE"])

In [ ]:
multiplier_model_summary.shape

In [ ]:
multiplier_model_summary.head()

In [ ]:
well_aligned_lvs = multiplier_model_summary[
    (
        multiplier_model_summary["FDR"] < 0.05
    )  # & (multiplier_model_summary["AUC"] >= 0.75)
]

display(well_aligned_lvs.shape)
display(well_aligned_lvs.head())

In [ ]:
well_aligned_lv_codes = set([f"LV{lvi}" for lvi in well_aligned_lvs["LV index"]])

In [ ]:
len(well_aligned_lv_codes)

In [ ]:
list(well_aligned_lv_codes)[:5]

# Select LVs from CRISPR analysis

In [ ]:
# FIXME: there will be a specific folder for crispr analysis in the future, that should be replaced here
deg_enrich = pd.read_csv(
    Path(
        conf.RESULTS["BASE_DIR"],
        "crispr_analyses",
        "fgsea-hi_conf-all_lvs.tsv",
    ).resolve(),
    sep="\t",
)

In [ ]:
deg_enrich.shape

In [ ]:
deg_enrich.head()

In [ ]:
# deg_enrich_max_idx = deg_enrich.groupby(["lv", "pathway"])["pval"].idxmax()

In [ ]:
# deg_enrich = deg_enrich.loc[deg_enrich_max_idx].reset_index(drop=True)
# display(deg_enrich.shape)
# display(deg_enrich.head())

In [ ]:
deg_enrich = deg_enrich.assign(
    lv_aligned=deg_enrich["lv"].apply(lambda x: x in well_aligned_lv_codes)
)

In [ ]:
deg_enrich = deg_enrich[(deg_enrich["lv_aligned"])]

In [ ]:
deg_enrich.shape

## Lipids-increasing gene sets

In [ ]:
deg_increase = deg_enrich[
    deg_enrich["pathway"].isin(("gene_set_increase",)) & (deg_enrich["pval"] < 0.01)
].sort_values("pval", ascending=True)

In [ ]:
deg_increase.shape

In [ ]:
deg_increase.head()

In [ ]:
lvs_increase = deg_increase["lv"].unique()

In [ ]:
lvs_increase.shape

In [ ]:
lvs_increase

## Lipids-decreasing gene sets

In [ ]:
deg_decrease = deg_enrich[
    deg_enrich["pathway"].isin(("gene_set_decrease",)) & (deg_enrich["pval"] < 0.01)
].sort_values("pval", ascending=True)

In [ ]:
deg_decrease.shape

In [ ]:
deg_decrease.head()

In [ ]:
lvs_decrease = deg_decrease["lv"].unique()

In [ ]:
lvs_decrease.shape

In [ ]:
lvs_decrease

## Merge into one dataframe

In [ ]:
_tmp0 = pd.DataFrame({"lv": lvs_increase, "lv_set": "lipids-increasing"})

_tmp1 = pd.DataFrame({"lv": lvs_decrease, "lv_set": "lipids-decreasing"})

In [ ]:
gls_selected_lvs = pd.concat([_tmp0, _tmp1], ignore_index=True)

In [ ]:
gls_selected_lvs.shape

In [ ]:
gls_selected_lvs.head()

# Select traits from main clusters

For this run on the LVs related to the lipids CRISPR analysis, I'm only interested in the main clusters of the cardiovascular sub-branch.

In [ ]:
k = 29

In [ ]:
part29 = pd.Series(best_partitions.loc[k, "partition"])

In [ ]:
part29.value_counts()

In [ ]:
part29_clusters = part29.unique()

In [ ]:
part29_clusters

In [ ]:
# number of traits?
part29.value_counts().loc[[x for x in part29_clusters if x not in (0,)]].sum()

In [ ]:
PHENOTYPES_CONFIG = [(k, x) for x in part29_clusters if x not in (0,)]

# GLSPhenoplier

## Get list of phenotypes/lvs pairs

In [ ]:
phenotypes_lvs_pairs = []

for part_k, cluster_id in PHENOTYPES_CONFIG:
    # get traits from the partition/cluster
    part = best_partitions.loc[part_k, "partition"]
    cluster_traits = data.index[part == cluster_id]

    for phenotype_code in cluster_traits:
        for idx, lv_row in gls_selected_lvs.iterrows():
            phenotypes_lvs_pairs.append(
                {
                    "phenotype_part_k": part_k,
                    "phenotype_cluster_id": cluster_id,
                    "phenotype": phenotype_code,
                    "lv": lv_row["lv"],
                    "lv_set": lv_row["lv_set"],
                }
            )

phenotypes_lvs_pairs = pd.DataFrame(phenotypes_lvs_pairs).drop_duplicates()

In [ ]:
phenotypes_lvs_pairs = phenotypes_lvs_pairs.sort_values(
    ["phenotype", "lv"]
).reset_index(drop=True)

In [ ]:
phenotypes_lvs_pairs.shape

In [ ]:
phenotypes_lvs_pairs.head()

## Run

In [ ]:
results = []

pbar = tqdm(total=phenotypes_lvs_pairs.shape[0])

for idx, row in phenotypes_lvs_pairs.iterrows():
    phenotype_code = row["phenotype"]
    lv_code = row["lv"]

    pbar.set_description(f"{phenotype_code} - {lv_code}")

    gls_model = GLSPhenoplier(
        smultixcan_result_set_filepath=conf.PHENOMEXCAN[
            "SMULTIXCAN_EFO_PARTIAL_MASHR_ZSCORES_FILE"
        ]
    )
    gls_model.fit_named(lv_code, phenotype_code)
    res = gls_model.results

    results.append(
        {
            "part_k": row["phenotype_part_k"],
            "cluster_id": row["phenotype_cluster_id"],
            "phenotype": phenotype_code,
            "lv": lv_code,
            "lv_set": row["lv_set"],
            "lv_with_pathway": lv_code in well_aligned_lv_codes,
            "coef": res.params.loc["lv"],
            "pvalue": res.pvalues_onesided.loc["lv"],
            "pvalue_twosided": res.pvalues.loc["lv"],
            "summary": gls_model.results_summary,
        }
    )

    # save results every 10 models trained
    if (idx % 10) == 0:
        pd.DataFrame(results).to_pickle(OUTPUT_FILENAME)

    pbar.update(1)

pbar.close()

In [ ]:
results = pd.DataFrame(results)

In [ ]:
results.shape

In [ ]:
results.head()

In [ ]:
results.sort_values("pvalue").head(10)

## Save

In [ ]:
results.to_pickle(OUTPUT_FILENAME)